<h3>機械学習課題 3 : Linear Regression 回帰分析 Part 2 - Gradient Descent 最急降下法</h3><br/>

<h4>注意 : pythonコードを実行しながら読んでください!</h4><br/>

前回の課題を一度復習しましょう。

前回は回帰分析(Linear Regression)を手動で実施してみました。目的関数$J$ (Cost Function)というのを作成し、教師データと線形方程式間の平均距離$J(\theta_0,\theta_1)$を測定しながら、関連パラメーターの変化による方程式のグラフの変化について実際にテストしてみました。

回帰分析をコンピューターに頼むために（機械学習を完成するために）は、あともう一つ残っています。具体的には最急降下法(Gradient Descent)という手法を使って、最小の$J(\theta_0,\theta_1)$をコンピューターに見つけてもらうことです。

まず、そもそも$\theta_0$と$\theta_1$が変化していくと$J(\theta_0,\theta_1)$がどう変わっていくのか可視化してみましょう。

最初は前回の課題でも使っていた教師データを読み込みましょう。

In [ ]:
#import required libraries
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#read learning data
data = np.loadtxt('ex1data1.txt', delimiter=',')

X = data[:,0]
Y = data[:,1] 

あと、前回作った目的関数Cost Functionを持ってきてください。

<h4>注意：</h4>

- 以下の部分のみコードを書いてください。他の部分は修正しないでください。

#------- Coding Start -------
#------- Coding End -------

In [ ]:
# Cost Function
def costFunction(X, Y, theta0, theta1):
    J = 0
    
    # ------- Coding Start -------

    # ------- Coding End -------
    return J

教師データと目的関数が準備できましたので、実際に$\theta_0$の値を-10から10まで、$\theta_1$は-1から4までの範囲で変化させながら、$J(\theta_0,\theta_1)$の値の変化をグラフで描いてみます。

In [ ]:
#create a meshgrid of theta0 and theta1
T0 = np.linspace(-10, 10, 50) # range of theta0 from -10 to 10
T1 = np.linspace(-1, 4, 50) # range of theta1 from -1 to 4
xx, yy = np.meshgrid(T0, T1, indexing='xy')

#get Cost J for all theta0 and theta1
zz = np.zeros((T0.size, T1.size))
for (i,j),v in np.ndenumerate(zz):
    zz[i,j] = costFunction(X, Y, xx[i,j], yy[i,j])

#draw 3D graph
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.plot_surface(xx, yy, zz, rstride=1, cstride=1, alpha=0.6, cmap=plt.cm.jet)
ax.set_xlabel(r'$\theta_0$')
ax.set_ylabel(r'$\theta_1$')
ax.set_zlabel(r'J($\theta_0$,$\theta_1$)')
ax.set_zlim(zz.min(),zz.max())
ax.view_init(elev=15, azim=230)

<h4>面白いグラフが表示されましたか。</h4>

我々に必要な情報は$J(\theta_0,\theta_1)$の最小の値でしたが、このグラフから見えますか。おおお、なるほど、多分青いところの何処かですよね。

ちなみに、皆さん、このグラフを見て何を思い出しましたか。

私はこのグラフからスキー場を思い出しました。赤いところから一番低いところまで滑っている自分の姿を思い出しました。実は最急降下法(Gradient Descent)というのもスキー場で滑っている人達と同じです。上の何処かから滑り始め、もう滑れないところまで滑っていく感じです。もう滑れないところに到着したら、そこの$J$って多分一番小さくて、その時の$\theta_0$と$\theta_1$の値で、教師データのパターンに一番当てはまる線形方程式が作れる意味になります。

そしたら、最急降下法(Gradient Descent)をどうやって実装すれば良いでしょうか。簡単にロジックを説明します。スキー場を滑っていると想像しながら読んでみましょう。

1. グラフ上の立って、そこから移動する（滑る）目的地のポイントを決める。
2. 現在の位置から目的地までの傾きを計算し、その分移動する（滑る）
3. 1、2を反復する。ただし、傾きがゼロになったら、もう滑れないので止める。

<h4>いかがでしょうか。</h4>

簡単でしょう。ステップ3で傾きがゼロになったらそのポイントが我々が探していた$J$が一番小さくなるところでしょう。ちなみに、実際に最急降下法を適用してみると傾きが完全にゼロにならないケースが多いです。なぜかというと、ステップ1で決まる移動距離によっては、ゼロになるところを通りすぎてしまう可能性があるからです。

移動距離を長くすると、最小の傾きを探すのに時間は短縮できますが、完全な最小の値が見つからない可能性があり、その反面、移動距離を短くすると、滑っていくのに相当な時間がかかってしまう、というメリットとデメリットがあります。

実際に最急降下法を適用する際に、この移動距離に相当する値を学習率(Learning Rate)と言います。記号は主に$\alpha$を使います。実際に機械学習を応用する際には、推論モデルの正答率と機械学習の性能の間のバランスを見て$\alpha$の値をチューニングしていく必要があります。

<h4>これから一番難しいところに入ります。それは、ステップ2でグラフの傾きをどう計算するかです。準備できましたか。</h4>

ここで、高校時代に勉強したことのある数学理論を持ってきて使ってみましょう。それは。。。

微分というやつです。<h4>微分!</h4> <h3>微分!</h3> <h2>微分!</h2> <h1>微分!</h1>

ここで諦めたくなる方が大勢いらっしゃると思いますが、今まで頑張ってきたのが勿体無いので、皆さん前向きに進めると思います。続きます。^_^

ちょっと、説明が長くなったので、ここでちょっと一休みしましょう。

<h4>簡単な課題を出します。（ちなみに、この課題はオプションですので、やらなくても良いです。）</h4>

先ほど、$\theta_0$、$\theta_1$、$J(\theta_0,\theta_1)$の関係を3次元のグラフで表わしてみましたが、実は2次元の表現も可能です。

matplotlibライブラリが提供するcontourグラフ表示機能を使って2次元グラフを描いてみましょう。

https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.contour.html

ちなみに、levelsパラメーターは$np.logspace(-2, 3, 20)$、cmapパラメーターは$plt.cm.jet$を使うともっと見やすいです。

<h4>注意：</h4>

- 以下の部分のみコードを書いてください。他の部分は修正しないでください。

#------- Coding Start -------
#------- Coding End -------

In [ ]:
T0 = np.linspace(-10, 10, 50)
T1 = np.linspace(-1, 4, 50)

xx, yy = np.meshgrid(T0, T1, indexing='xy')
zz = np.zeros((T0.size, T1.size))

for (i,j),v in np.ndenumerate(zz):
    zz[i,j] = costFunction(X, Y, xx[i,j], yy[i,j])
    
fig_op = plt.figure(figsize=(6,6))
ax_op = fig_op.add_subplot(111)

#------- Coding Start -------

#------- Coding End -------

ax_op.set_xlabel(r'$\theta_0$')
ax_op.set_ylabel(r'$\theta_1$')

<h4>いかがでしょうか。もう少し分かりやすくなりましたか。</h4>

前回の最後の課題で皆さんが決めていた$\theta_0$と$\theta_1$の値と上のグラフの中心の値を比較してみましょう。面白い発見があると思います。かなり近いでしょう。

今回の課題がちょっと長くなってしまったので、本格的な最急降下法(Gradient Descent)の実装、具体的には微分への取り組みは次回にします。

<h4>お疲れ様でした。</h4>